---
# **Roberta**
---
Link: [ TweetEval benchmark ](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest)

Sentiment Categories:
positive 😊
negative 🙁
neutral 😐

In [1]:
!pip install transformers
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.9/981.9 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [2]:
!pip install sentencepiece

In [3]:
!pip install demoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.2 MB/s eta 0:00:00


In [4]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 6.5 MB/s eta 0:00:00


In [69]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
import pandas as pd
import re
import emoji
import demoji

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [70]:
def emoji_free_text(text):
    allchars = [str for str in text.decode('utf-8')]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.decode('utf-8').split() if not any(i in str for i in emoji_list)])
    return clean_text

In [71]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [72]:
def filter(post):
  try:
    post = re.sub(r"^https://ibb.co/[a-zA-Z0-9]*\s", " ", post)
    post = re.sub(r"\s+https://ibb.co/[a-zA-Z0-9]*\s", " ", post)
    post = re.sub(r"(https://ibb.co/[a-zA-Z0-9]*)", " ", post)
    post = re.sub(r"(https://www.reddit.com/r/[a-zA-Z0-9]*)", " ", post)
    post = re.sub(r"(https://[a-zA-Z0-9//._@-]*)", " ", post)
    post = re.sub(r"\s+https://ibb.co/[a-zA-Z0-9]*$", " ", post)
    post = re.sub(r"\s+https://ibb.co/[a-zA-Z0-9]*$", " ", post)
    post = re.sub(r"(https://cbcn.bandcamp.com/[a-zA-Z0-9//]*)", " ", post)
    post = re.sub("@[A-Za-z0-9_]+","", post)
    post = re.sub("#[A-Za-z0-9_]+","", post)
    post = re.sub(r'//www.reddit.com/r/[a-zA-Z0-9]*\s', ' ', post)
    # post = clean(post.encode('utf8'), no_emoji=True)
    # post = emoji.replace_emoji(post, replace="")
    post = re.sub(r"that's","that is", post)
    post = re.sub(r"there's","there is", post)
    post = re.sub(r"what's","what is", post)
    post = re.sub(r"where's","where is", post)
    post = re.sub(r"it's","it is", post)
    post = re.sub(r"It's","it is", post)
    post = re.sub(r"I'm","I am", post)
    post = re.sub(r"who's","who is", post)
    post = re.sub(r"i'm","i am", post)
    post = re.sub(r"she's","she is", post)
    post = re.sub(r"he's","he is", post)
    post = re.sub(r"you're","you are", post)
    post = re.sub(r"they're","they are", post)
    post = re.sub(r"who're","who are", post)
    post = re.sub(r"ain't","am not", post)
    post = re.sub(r"wouldn't","would not", post)
    post = re.sub(r"shouldn't","should not", post)
    post = re.sub(r"can't","can not", post)
    post = re.sub(r"couldn't","could not", post)
    post = re.sub(r"won't","will not", post)
    post = re.sub(r"don't","do not", post)
    post = re.sub(r"\s+[s]\s+"," ", post)
    post = re.sub(r"\s*[\[\]\(\)\*#<>\'\":]\s*"," ", post)
    post = re.sub(r"\s+"," ", post)
    post = deEmojify(post)
    post = demoji.replace(post)
  except TypeError:
      pass
  return post

In [73]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [74]:
text = """i’m a 15 year old boy who was diagnosed with lung cancer 2 weeks ago and almost immediately started chemotherapy. when i got diagnosed i explained to my parents, 44 year old mom and 48 year old dad that i’m aware that i might fall back into depression because of the circumstances but i don’t want them to do anything like shaving their hair as i know at least that my dad’s hair isn’t gonna grow bag and i can’t even imagine that i caused my dad’s chance of regrowing hair to deplete completely( it sounds silly but i love his tiny head of hair lol) i’m scared they’ll try something like that… how can i convince them not to do that? please, i don’t need sympathy, i accepted that cancer sucks lol<3.

edit: thanks for all the replies, i left out a bit of information, first, my lung cancer is stage 1. i have no idea how i just felt something was wrong that i went to the doctor where they diagnosed me, also this isn’t the first time i’ve been diagnosed with cancer, i was diagnosed with stage 1 melanoma in 2020 at 12 years old s but luckily they cut it out immediately and i just needed to go for checkups for like 2 years but they couldn’t find my lung cancer.

also i went for my chemo today and the cancer has reduced in size by 20% which is amazing, my doctor have never seen lung cancer reducing so fast in someone my age lol, so road to recovery is going great. i started losing my eyebrows today but with the rate of chemo working, i’ll hopefully be cancer free with a half head of hair lol"""

In [75]:
posts_emo = {}
sentences = nltk.sent_tokenize(text)
for sent in sentences:
    if(sent):
      sent = filter(sent)
      text = preprocess(sent)
      encoded_input = tokenizer(text, return_tensors='pt')
      output = model(**encoded_input)
      scores = output[0][0].detach().numpy()
      scores = softmax(scores)
      ranking = np.argsort(scores)
      ranking = ranking[::-1]
      sentiment_scores = []
      for j in range(scores.shape[0]):
            l = model.config.id2label[ranking[j]]
            s = scores[ranking[j]]
            # print(f"{text}{i+1}) {l} {np.round(float(s), 4)}")
            sentiment = l + '=' + str(np.round(float(s), 4))
            sentiment_scores.append(sentiment)
      posts_emo[sent] = sentiment_scores

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [76]:
posts_emo

{'i’m a 15 year old boy who was diagnosed with lung cancer 2 weeks ago and almost immediately started chemotherapy.': ['negative=0.5255',
  'neutral=0.4296',
  'positive=0.0449'],
 'when i got diagnosed i explained to my parents, 44 year old mom and 48 year old dad that i’m aware that i might fall back into depression because of the circumstances but i don’t want them to do anything like shaving their hair as i know at least that my dad’s hair isn’t gonna grow bag and i can’t even imagine that i caused my dad’s chance of regrowing hair to deplete completely it sounds silly but i love his tiny head of hair lol i’m scared they’ll try something like that… how can i convince them not to do that?': ['negative=0.7073',
  'neutral=0.2704',
  'positive=0.0223'],
 'please, i don’t need sympathy, i accepted that cancer sucks lol 3.': ['negative=0.8323',
  'neutral=0.1527',
  'positive=0.015'],
 'edit thanks for all the replies, i left out a bit of information, first, my lung cancer is stage 1. i

In [82]:
posts_emotions = pd.DataFrame(columns = ['Post Text', 'Emotion Score'])

In [83]:
posts_emotions['Post Text'] = posts_emo.keys()
posts_emotions['Emotion Score'] =  posts_emo.values()

In [84]:
posts_emotions

,Post Text,Emotion Score
0,i’m a 15 year old boy who was diagnosed with l...,"[negative=0.5255, neutral=0.4296, positive=0.0..."
1,when i got diagnosed i explained to my parents...,"[negative=0.7073, neutral=0.2704, positive=0.0..."
2,"please, i don’t need sympathy, i accepted that...","[negative=0.8323, neutral=0.1527, positive=0.015]"
3,"edit thanks for all the replies, i left out a ...","[negative=0.5916, neutral=0.3837, positive=0.0..."
4,also i went for my chemo today and the cancer ...,"[positive=0.9666, neutral=0.0249, negative=0.0..."
5,i started losing my eyebrows today but with th...,"[positive=0.5477, neutral=0.3899, negative=0.0..."


End.

---